<a href="https://colab.research.google.com/github/bgcarcellar/UStAC-Codes/blob/main/bs_ge_courses_demand_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install google-colab-selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.8 MB/s eta 0:00:00


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import google_colab_selenium as gs
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#Grades Extractor

In [3]:
import re
import numpy as np
import json
import pandas as pd
import glob

In [23]:
def completedGrade(x):
  if str(x) == "P":
    return 0
  elif str(x) == "DRP":
    return 3.5

  try:
    grade = re.findall(r"[-+]?\d*\.\d+|\d+", str(x))[-1]
    return float(grade)
  except:
    return np.nan

In [24]:
def gen_sns(allfiles):
  snlist = []
  for f in allfiles:
    df = pd.read_csv(f,skiprows=9)
    df = pd.read_csv(f)
    f_list = list(df.STUDENTNO.unique())
    snlist += f_list
  snlist = list(set(snlist))
  # Convert snlist to a NumPy array
  snlist = np.array(snlist)
  # Use a boolean array to filter out NaN values
  snlist = snlist[~np.isnan(snlist)]
  # Convert snlist back to a list
  snlist = snlist.tolist()
  return snlist

In [25]:
class Subject:

  def __init__(self,name,units,prereqID = [], coreqID = [],alias = []):
    self.name = name
    self.units = units
    self.prereqID = prereqID
    self.coreqID = coreqID
    self.takes = 0
    self.grades = []
    self.prof = []
    self.currentProf = ''
    self.currentGrade = np.nan
    self.alias = alias
    self.prereq = []
    self.coreq = []

  def addCoreq(self,subj):
    self.coreq.append(subj)

  def addTake(self, prof,grade):
    self.prof.append(prof)
    self.grades.append(grade)
    self.takes += 1
    self.currentProf = prof
    self.currentGrade = grade

  def addPrereq(self,subj):
    self.prereq.append(subj)

  def getGrade(self):
    return sum(self.grades)/self.takes

  def __repr__(self):
    return self.name

In [26]:
class Curriculum:
  def __init__(self,json_file):
      with open(json_file, 'r') as file:
          data = json.load(file)
      self.name = data.get('name')
      self.year = data.get('year')
      self.subjects = self.populateSubj(data.get('subjects'))

      for subs in self.subjects.values():
        for i in subs.prereqID:
          subs.addPrereq(self.subjects[i])
        for i in subs.coreqID:
          subs.addCoreq(self.subjects[i])

  def populateSubj(self,subs):
    subjects = {}
    for i in subs:
      subj = Subject(i.get('title'),i.get('unit'), i.get('prereq'), i.get('coreq'), alias = i.get('alias'))
      subjects[i.get('id')] = subj
    return subjects

In [27]:
class Student:
  def __init__(self, f, curriculum_json):
    self.curriculum = Curriculum(curriculum_json)
    self.othersubs = []

    try:
      self.name, self.sn, self.subjects = self.openHTML(f)
      name_to_id = {i.name: id for id, i in self.curriculum.subjects.items()}
      for index, row in self.subjects.iterrows():
        subject_id = name_to_id.get(row['course'])
        if subject_id != None:
          self.curriculum.subjects[subject_id].addTake(row['Instructor/s'],row['Grade'])
        else:
          sub = Subject(row['course'],row['Units'])
          sub.addTake(row['Instructor/s'],row['Grade'])
          self.othersubs.append(sub)


      self.grades = pd.DataFrame({
          'course': [i.name for i in list(self.curriculum.subjects.values())],
          'takes': [i.takes for i in list(self.curriculum.subjects.values())],
          'current grade': [i.currentGrade if hasattr(i,'currentGrade') else None for i in list(self.curriculum.subjects.values())]
      })

    except:
      raise ValueError('Unsupported file type for f argument')

  def openHTML(self,f):
    grades = pd.read_html(f)
    master = pd.DataFrame({})
    name = grades[0].loc[0,0]
    sn = grades[0].loc[1,0]

    for i in range(1,len(grades)-5):
      try:
        sem = (grades[i].columns[0][0])
        grades[i].columns = [a[1] for a in  grades[i].columns.to_flat_index()]
        grades[i]['sem']=sem
        master = pd.concat([master,grades[i][['Class','Instructor/s','Units','Grade','sem']]])
      except:
        sem = (grades[i].columns[0][0])
        print(sem)
        continue
    print("Columns after reading HTML:", master.columns) # Added print statement
    master['Instructor/s'] = master['Instructor/s'].fillna('No Record')
    master['Grade'] = master['Grade'].fillna(6)
    master = master.dropna()
    master = master.reset_index()
    master[['course','section']] = master['Class'].str.extract(r'(\w+\s[\d.]+)\s(.*)')
    master['Grade'] = master.Grade.apply(completedGrade)
    master['Units'] = master.Units.apply(completedGrade)
    return name, sn, master

  def solveCluster(self):
    cluster = {
        'GC' : ['GE 120','GE 122','GE 127','GE 128','GE 129'],
        'G' : ['GE 161','GE 162','GE 168','GE 169','GE 151','GE 152','GE 153','GE 154','GE 155','GE 155.1','GE 155.2'],
        'S' : ['GE 10','GE 12','GE 116','GE 117','GE 118','GE 119','GE 193'],
        'GIS' : ['GE 30','GE 173','GIM 165','GIM 175','GIM 176','GIM 177'],
        'RS' : ['GsE 188','GsE 189'],
        'L' : ['GE 142','GE 143','GE 144','GE 148','GE 149'],
        'RP' : ['GE 2','GE 192','GE 197','GE 191','GE 190','GE 199','GE 200'],
        'MP' : ['Math 21','Math 22','Math 23','Math 17','Math 53','Math 54','Math 55','Physics 71','Physics 72','Physics 73','Physics 71.1','Physics 72.1','Physics 73.1'],
        'Gen': ['Kas 1','Arts 1','Speech 30','Fil 40','Philo 1','Soc Sci 2','Eng 13','Eng 13','STS 1']
    }
    clusterGrade = {}
    for i in cluster.keys():
      filter = self.grades[self.grades['course'].isin(cluster[i])]
      filter = filter.dropna()
      prof = list(set(filter['Instructor/s'].values))
      clusterGrade[i+'_prof'] = prof
      total_units = sum(filter.Units)
      if total_units != 0:
        clusterGrade[i]=sum(filter.Units*filter.Grade)/total_units
      else:
        clusterGrade[i] = np.nan
    return clusterGrade

  def checkPrereq(self,subject,overrides = []):
    subs = [i for i in self.curriculum.subjects.values() if i.name == subject ]
    if subs:
      prereq_grade = [j.currentGrade for j in subs[0].prereq if j.name not in overrides]
      coreq_prereq_grade =[]
      for j in subs[0].coreq:
        cp_grade = [k.currentGrade for k in j.prereq if k.name not in overrides]
        coreq_prereq_grade += cp_grade

      all_grades = prereq_grade + coreq_prereq_grade
      return (not np.any(np.isnan(all_grades))) and (not (np.any((np.array(all_grades) == 5) | (np.array(all_grades) == 4) | (np.array(all_grades) == 3.5))))

    else:
      return False

  def checkTaken(self, subject):
    subs = [i for i in self.curriculum.subjects.values() if i.name == subject ]
    if subs:
      if subs[0].takes > 0:
        return True
      else:
        return False
    else:
      return False

  def hasTakenAndPassed(self,subject):
    subs = [i for i in self.curriculum.subjects.values() if i.name == subject ]
    if subs:
      if subs[0].takes > 0:
        if subs[0].currentGrade <= 3:
          return True
        else:
          return False
      else:
        return False
    else:
      return False

  def isGraduate(self):
    course_to_remove = ['[PE]', 'NSTP 1','NSTP 2','[Elective]','[GE]']
    filtered_df = self.grades[~self.grades['course'].isin(course_to_remove)]
    filtered_df = filtered_df[filtered_df['current grade'].isnull()]
    print(filtered_df)
    return not filtered_df.isnull().values.any()

  def subsToTake(self):
    course_to_remove = ['[PE]', 'NSTP 1','NSTP 2','[Elective]','[GE]']
    filtered_df = self.grades[~self.grades['course'].isin(course_to_remove)]
    filtered_df = filtered_df[(filtered_df['current grade'].isnull()) | (filtered_df['current grade'] < 3)]
    return filtered_df

  def isOngoing(self):
    [i.currentGrade for i in list(self.curriculum.subjects.values())]
    return [i.name for i in list(self.curriculum.subjects.values()) if i.currentGrade==6]

In [28]:
class Department:
  def __init__(self, name, current_curriculum):
    self.name = name
    self.students = []
    self.student_graduates = []
    self.student_ongoing = []
    self.current_curriculum = Curriculum(current_curriculum)

  def addStudent(self,student):
    self.students.append(student)

  def checkDemand(self, subject,overrides = []):
    subs = [i for i in self.current_curriculum.subjects.values() if i.name == subject ]
    if subs:
      print(f'Checking prerequisites for {subs[0].name}')
      print('Prerequisites:')
      for j in subs[0].prereq:
        print(j.name)
      print('Corequisites:')
      for j in subs[0].coreq:
        print(j.name)
      for j in overrides:
        print(f'Overriding grades for {j}')

      filtered_students = [student for student in self.students if ((student.checkPrereq(subject,overrides)) and not (student.hasTakenAndPassed(subject)))]
      subs = [i for i in self.current_curriculum.subjects.values() if i.name == subject ][0]
      subs_keys = [i for i in self.current_curriculum.subjects.keys() if self.current_curriculum.subjects[i].name == subject][0]

      cp = [j.prereq for j in subs.coreq if j.name not in overrides]
      coreq_prereq = [item for sublist in cp for item in sublist]
      all_subs = list(set(coreq_prereq + subs.prereq))

      subj_df = pd.DataFrame({
          'name': [i.name for i in filtered_students],
          'sn': [i.sn for i in filtered_students],
          'takes': [i.curriculum.subjects[subs_keys].takes for i in filtered_students],
          'current grade': [i.curriculum.subjects[subs_keys].currentGrade for i in filtered_students],
          'prereq': [i.checkPrereq(subject) for i in filtered_students],
      })

      for subj in all_subs:
        subj_df[str(subj.name)] = [i.grades[i.grades['course'] == subj.name]['current grade'].values[0] for i in filtered_students]

      for subj in overrides:
        subj_df[subj + "_ongoing"] = [(subj in i.isOngoing()) for i in filtered_students]

      return(subj_df)
    else:
      print(f'No subject {subject} found in department current curriculum')


# Demand Generation

In [29]:
dge = Department('DGE','/content/BSGE_2019 (1).json')
sn_list = gen_sns(glob.glob('/content/*.csv'))

In [30]:
# Step 1: Define the login URL and payload
login_url = 'https://crs.upd.edu.ph/'
target_url = 'https://crs.upd.edu.ph/viewgrades/'
payload = {
    'username': 'dge',
    'password': 'updgecrs1937',
    # Include any additional fields required by the login form
}

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Set up ChromeDriver
service = Service('/usr/local/bin/chromedriver')
driver = gs.Chrome()

wait = WebDriverWait(driver, 30)
driver.get(login_url)
wait.until(EC.element_to_be_clickable((By.NAME, "txt_login"))).send_keys(payload['username'])

password_field = driver.find_element(By.NAME, 'pwd_password')
password_field.send_keys(payload['password'])

# Submit the login form
password_field.send_keys(Keys.RETURN)
print("Logged in")

# Navigate to the target URL
driver.get(target_url)
error_sns = []

for studentno in sn_list:
  # Find the text box element (adjust the locator as needed)
  text_box = driver.find_element(By.ID,'txt_studentno')
  # Enter a value into the text box
  text_box.send_keys(str(studentno))
  text_box.submit()

  # Optionally, wait for the page to load or for certain elements to appear
  driver.implicitly_wait(10)

  page_source = driver.page_source
  try:
    student = Student(page_source, '/content/BSGE_2019 (1).json')
  except:
    print(studentno)
    error_sns.append(studentno)
    continue
  dge.addStudent(student)
  break

<IPython.core.display.Javascript object>

Logged in


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


202313733


In [31]:
# Step 1: Define the login URL and payload
login_url = 'https://crs.upd.edu.ph/'
target_url = 'https://crs.upd.edu.ph/viewgrades/'
payload = {
    'username': 'dge',
    'password': 'updgecrs1937',
    # Include any additional fields required by the login form
}

# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Set up ChromeDriver
service = Service('/usr/local/bin/chromedriver')
driver = gs.Chrome()

wait = WebDriverWait(driver, 30)
driver.get(login_url)
wait.until(EC.element_to_be_clickable((By.NAME, "txt_login"))).send_keys(payload['username'])

password_field = driver.find_element(By.NAME, 'pwd_password')
password_field.send_keys(payload['password'])

# Submit the login form
password_field.send_keys(Keys.RETURN)
print("Logged in")

# Navigate to the target URL
driver.get(target_url)
error_sns = []

def try_load_students(student_list):
    new_errors = []
    for studentno in student_list:
        try:
            text_box = driver.find_element(By.ID, 'txt_studentno')
            text_box.clear()
            text_box.send_keys(str(studentno))
            text_box.submit()

            driver.implicitly_wait(10)
            page_source = driver.page_source

            student = Student(page_source, '/content/BSGE_2019 (1).json')
            dge.addStudent(student)
        except Exception as e:
            print(f"❌ Error loading {studentno}")
            new_errors.append(studentno)
    return new_errors

# Initial attempt
error_sns = try_load_students(sn_list)

# Retry loop
max_retries = 5  # prevent infinite loops
attempt = 1

while attempt <= max_retries:
    print(f"\n🔁 Retry Attempt {attempt} — Errors remaining: {len(error_sns)}")

    new_errors = try_load_students(error_sns)

    if set(new_errors) == set(error_sns):
        print("✅ No changes in error list. Stopping retries.")
        break
    else:
        print(f"ℹ️ Students recovered this round: {set(error_sns) - set(new_errors)}")
        error_sns = new_errors
        attempt += 1

print("\n🏁 Done. Final students with errors:")
print(error_sns)


<IPython.core.display.Javascript object>

Logged in


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


❌ Error loading 202313733


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


...

❌ Error loading 202303240
❌ Error loading 202401554
❌ Error loading 202200863


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


❌ Error loading 202413930


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


❌ Error loading 202409889
❌ Error loading 202409891
❌ Error loading 202401735
ℹ️ Students recovered this round: {202211456, 202404192, 202303234, 202305411, 202400279, 202312518, 202401929, 202200506, 202302227, 202221207, 202308088, 202212506}

🏁 Done. Final students with errors:
[202412050, 202405915, 202407970, 202408004, 202311762, 202305683, 202311836, 202410184, 202209499, 202402026, 201490729, 202414387, 202201399, 202404191, 202307951, 202416506, 202402184, 202408357, 202303941, 202303942, 202303947, 202111451, 202420702, 202308103, 202404377, 202308124, 202301991, 202410579, 202404453, 202304104, 202306169, 202306178, 202310300, 202300083, 202412739, 202111689, 202316500, 202314501, 202404614, 202320668, 202404658, 202408792, 202300252, 202306414, 202404724, 202412935, 202402719, 202408993, 202304590, 202402917, 202404990, 202321040, 202304657, 202409117, 202317089, 202401064, 202300715, 202306864, 202401095, 202212713, 202200429, 202403260, 202311100, 202200511, 202403313, 20

In [50]:
import time

# Customize how many times to try each student individually
individual_retries = 3

# List of students who still failed after batch retries
final_errors = []

for studentno in error_sns:
    success = False

    for attempt in range(1, individual_retries + 1):
        try:
            print(f"🔄 Retrying {studentno} (Attempt {attempt})")

            # Clear and enter student number
            text_box = driver.find_element(By.ID, 'txt_studentno')
            text_box.clear()
            text_box.send_keys(str(studentno))
            text_box.submit()

            # Wait and fetch page content
            driver.implicitly_wait(10)
            page_source = driver.page_source

            # Try to load student
            student = Student(page_source, '/content/BSGE_2019 (1).json')
            dge.addStudent(student)
            print(f"✅ Successfully loaded: {studentno}")
            success = True
            break  # Exit retry loop if successful

        except Exception as e:
            print(f"⚠️ Attempt {attempt} failed for {studentno}: {e}")
            time.sleep(2)  # wait before retrying

    if not success:
        final_errors.append(studentno)

# Final report
print("\n📋 Students still failed after individual retries:")
print(final_errors)

🔄 Retrying 202407970 (Attempt 1)


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


⚠️ Attempt 1 failed for 202407970: Unsupported file type for f argument
🔄 Retrying 202407970 (Attempt 2)


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


⚠️ Attempt 2 failed for 202407970: Unsupported file type for f argument
🔄 Retrying 202407970 (Attempt 3)


...

⚠️ Attempt 1 failed for 202200863: Unsupported file type for f argument
🔄 Retrying 202200863 (Attempt 2)


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


⚠️ Attempt 2 failed for 202200863: Unsupported file type for f argument
🔄 Retrying 202200863 (Attempt 3)


/tmp/ipython-input-3295442611.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  grades = pd.read_html(f)


⚠️ Attempt 3 failed for 202200863: Unsupported file type for f argument

📋 Students still failed after individual retries:
[202407970, 202111451, 202111689, 202320668, 202200429, 202303088, 202200863]


In [51]:
error_sns = final_errors

# Demand Report

In [52]:
ay = '24-25'

In [53]:
print(f'The number of students in this is analysis is {len(dge.students)}.')
print(f'This number is based on the list of enrollees in the first and 2nd semester of AY {ay}, which has a total number of unique enrollments of {len(sn_list)}')

The number of students in this is analysis is 301.
This number is based on the list of enrollees in the first and 2nd semester of AY 24-25, which has a total number of unique enrollments of 307


##Second Year Courses

In [54]:
all_cluster_grades = []
for student in dge.students:
  cluster_grade = student.solveCluster()
  cluster_grade['name'] = student.name
  cluster_grade['sn'] = student.sn
  all_cluster_grades.append(cluster_grade)

cluster_grades_df = pd.DataFrame(all_cluster_grades)
display(cluster_grades_df.head())

KeyError: 'Instructor/s'

###GE 117

In [34]:
subject = "GE 149"
overrides = []

ge_subject = dge.checkDemand(subject,overrides)

Checking prerequisites for GE 149
Prerequisites:
Corequisites:


In [35]:
ge_subject

,name,sn,takes,current grade,prereq
0,"ERONICO, ANGELA MAE, ABELLERA",2024-14091,0,NaN,True
1,"ZULUETA, IVAN PAUL, PORTALEZA",2023-13733,0,NaN,True
2,"ERONICO, ANGELA MAE, ABELLERA",2024-14091,0,NaN,True
3,"PLEÑOS, JOSE EDUARDO, ARRADAZA",2024-20241,0,NaN,True
4,"DE ASIS, PRECIOUS LAHRA,",2024-12050,0,NaN,True
...,...,...,...,...,...
143,"TOMULTO, RALPH DANIEL, SUNGA",2024-01095,0,NaN,True
144,"ESPINOZA, SOFIA CAMILLE, ZABALA",2023-00789,0,NaN,True
145,"UGABAN, GIAN KARLA, MORENO",2022-03345,0,NaN,True
146,"RICAFORT, JEC, HIDALGO",2024-00017,0,NaN,True


In [ ]:
dge.students[0].subjects

,index,Class,Instructor/s,Units,Grade,sem,course,section
0,0,GE 2 1AB,"MEDINA, JOMMER",1.0,1.00,First Semester AY 2022-2023,GE 2,1AB
1,1,GE 10 1B,"VERGARA, KARL ADRIAN",3.0,2.25,First Semester AY 2022-2023,GE 10,1B
2,2,KAS 1 THW2,"LLANES, FERDINAND",3.0,1.25,First Semester AY 2022-2023,KAS 1,THW2
3,3,Math 21 TWHFR-9,"MANALOTO, KORINA ERNJULIE",4.0,5.00,First Semester AY 2022-2023,Math 21,TWHFR-9
4,4,Physics 71 TWHFU-5,"NIP, NIP; CABANILLA, JAYSON",4.0,5.00,First Semester AY 2022-2023,Physics 71,TWHFU-5
5,5,Physics 71.1 WIJ-1,"AGAPITO, XYD",1.0,1.25,First Semester AY 2022-2023,Physics 71.1,WIJ-1
6,6,Soc Sci 2 THX-1,"ESTRADA, LUIS GABRIEL ALFONSO",3.0,1.75,First Semester AY 2022-2023,Sci 2,THX-1
7,0,GE 12 1B,"CRUZ, MARGAUX ANGELICA",4.0,1.75,Second Semester AY 2022-2023,GE 12,1B
8,1,GE 30 1B,"ESCOTO, JOHN EMMANUEL",3.0,1.50,Second Semester AY 2022-2023,GE 30,1B
9,2,GE 120 1B,"PADON, RANEL",3.0,2.00,Second Semester AY 2022-2023,GE 120,1B


In [ ]:
[i.name for i in list(dge.students[0].curriculum.subjects.values()) if i.currentGrade==6]

['GE 144', 'GE 116', 'GIM 176', 'GsE 188', 'GsE 189']

In [ ]:
dge.students[0].isOngoing()

[2.5, 1.25, 3.0, 1.0, 2.25, nan, nan, nan, 1.75, 2.0, 1.75, 1.25, 1.5, 2.75, 2.75, nan, 2.75, 3.0, 1.5, 3.0, 2.75, 1.75, 2.0, 2.25, 2.0, 1.25, 2.75, 2.75, 1.0, 6.0, 6.0, 6.0, 6.0, nan, 6.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]


['GE 144', 'GE 116', 'GIM 176', 'GsE 188', 'GsE 189']

In [ ]:
[ for i in dge.students[0].curriculum.subjects.values()][0].takes

2

In [ ]:
[i.name for i in dge.students[0].curriculum.subjects.values() if i.currentGrade == 6]

['GE 144', 'GE 116', 'GIM 176', 'GsE 188', 'GsE 189']

In [32]:
all =[]
profs = []
for student in dge.students:
  clustergrade = student.solveCluster()
  clustergrade['name'] = student.name
  clustergrade['sn'] = student.sn
  all.append(clustergrade)
  #profs.append(clusterprofs)

ZeroDivisionError: division by zero

In [ ]:
all_df = pd.DataFrame(all)
all_df.to_csv('clustergrade.csv')

In [ ]:
student = Student(f)
clustergrade = student.solveCluster()
clustergrade['name'] = student.name
clustergrade['sn'] = student.sn
all.append(clustergrade)
profs.append(clusterprofs)

In [33]:
files = glob.glob('/content/drive/MyDrive/DGE_Files/UStAC/BSGE2023 Grades/*.html')
all =[]
profs = []
for f in files:
  student = Student(f)
  clustergrade = student.solveCluster()
  clustergrade['name'] = student.name
  clustergrade['sn'] = student.sn
  all.append(clustergrade)
  profs.append(clusterprofs)